In [ ]:
# 1. translate russian test through foo_dict
# 2. merge duplicates in test
# 3. remove cycle cases like ['design', 'design'] in test
# 4. create en foo on the base of foo_dict
# 5. remove duplicates in new foo

In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
# кодирую файл связей на основе словаря, далее на основе этой разметки можно делать замены
# в словаре содержатся русские слова, стобец количества связей и английский перевод
foo = pd.read_csv('210216_arctic_foo_ruen_dict_e2e.csv', sep=';', names = ["w1", 'value', "w2"])
test = pd.read_csv('arctic_RUS_test.csv', names = ["w1", "w2", 'value'])

display(foo[foo['w2'].duplicated()])

,w1,value,w2
9,проектирование,312,design
17,проектный,212,design
39,транспорт,152,transport
72,новое,108,new
104,основный,81,main
118,туристический,74,touristic
121,окружающий,73,environmental
132,иной,69,different
139,технология,66,technology
146,приложение,63,application


In [3]:

# 1.
# здесь делается перекодировка файла test на основе словаря foo,
# т.е. замена русских слов в файле tests на переводы из foo. 
# foo д.б. словарем - т.е. русские и их перевод

# create columns for translate
test.loc[:, 'translate_w1']  = np.nan
test.loc[:, 'translate_w2']  = np.nan

# encoding first word from edge list
# не кодируем а сразу заеняем на перевод, т.к. на входе словарь
for i in range(len(foo)):
    if not(test[test.iloc[:,0] == foo.iloc[i,0]]).empty:
        # set num of i in new col of test
        test.iloc[:,3] = np.where((test.iloc[:,0]==foo.iloc[i,0]), foo.iloc[i,2], test.iloc[:,3])

# удаляем все позиции оставшимися NaN-ми, это значит, что их нет в foo
testna = test[pd.isna(test.iloc[:,3])]
test = test[pd.notna(test.iloc[:,3])]

# encoding second word from edge list
for i in range(len(foo)):
    if not(test[test.iloc[:,1] == foo.iloc[i,0]]).empty:
        # set num of i in new col of test
        test.iloc[:,4] = np.where((test.iloc[:,1]==foo.iloc[i,0]), foo.iloc[i,2], test.iloc[:,4])

testna = test[pd.isna(test.iloc[:,4])]
test = test[pd.notna(test.iloc[:,4])]

# clean edge list
print(len(test))
print(test.iloc[0:10,])


15998
               w1             w2  value     translate_w1   translate_w2
0   существование       характер      1        existence      character
1         высокий        техника      1             high     technology
2  индивидуальный  экологический      4       individual  environmental
3     возможность    технический      4      opportunity  technological
4        комплекс           опыт      2          complex     experience
5          модель        элемент      3            model        element
6             год        научный      4             year     scientific
7        вахтовый        влияние      4  fly-in-fly-out          impact
8           образ    организация      5            image   organization
9           жизнь         знание      2             life      knowledge


In [5]:
# is there 0 from begining?
display(test[test['value'] == 0])


,w1,w2,value,translate_w1,translate_w2


In [53]:
# 2.
# try to merge duplicates in test

#test_n = pd.read_csv('210215_arctic_ruen_test.csv', names = ["w1", "w2", 'value'])
test_d = test.iloc[:,[3, 4, 2]]
test_d["duplicates"] = test_d.iloc[:,0:2].duplicated(keep = False)
# A value is trying to be set on a copy of a slice from a DataFrame.
# попробуй сперва инициализировать
display(test_d[test_d.iloc[:,0:2].duplicated(keep = False)])
test_d
for i in range(0,len(test_d)):
    if test_d.iloc[i, 3] == True:
        # нужно найти все дубликаты, сложить числа в ячейке value
        # сложтить их к первому дубликату, а остальные дубликаты удалить
        t = test_d.iloc[i,0:2]
        for j in range(i+1,len(test_d)):
            if test_d.iloc[j, 3] == True:
                if (list(t) == list(test_d.iloc[j,0:2])): 
                    #print(i, j, test_d.iloc[i,0], test_d.iloc[i,1], test_d.iloc[j,0], test_d.iloc[j,1], test_d.iloc[i,2], test_d.iloc[j,2], test_d.iloc[i, 2] + test_d.iloc[j, 2])
                    test_d.iloc[i, 2] = test_d.iloc[i, 2] + test_d.iloc[j, 2]
                    test_d.iloc[j, 2] = 0
test_d 


<ipython-input-53-dd9ddf469d2e>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_d["duplicates"] = test_d.iloc[:,0:2].duplicated(keep = False)


,translate_w1,translate_w2,value,duplicates
1,high,technology,1,True
17,level,extreme,2,True
21,main,special,1,True
26,given,scholarship,2,True
29,opportunity,main,5,True
...,...,...,...,...
15982,design,resource,4,True
15987,material,expedition,6,True
15990,transport,function,2,True
15995,example,extreme,2,True


C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


1 2844 high technology high technology 1 2 3
17 5888 level extreme level extreme 2 2 4
21 11130 main special main special 1 2 3
26 4127 given scholarship given scholarship 2 5 7
29 9651 opportunity main opportunity main 5 2 7
31 2433 context new context new 2 13 15
33 14588 scholarship ritual scholarship ritual 1 1 2
39 7940 year technology year technology 4 3 7
42 4446 ritual element ritual element 3 1 4
48 8971 given system given system 2 6 8
56 5395 given ornament given ornament 9 3 12
60 10983 situation extreme situation extreme 6 4 10
61 14803 special touristic special touristic 1 3 4
65 8557 image main image main 5 3 8
68 3216 action design action design 4 2 6
68 11919 action design action design 6 3 9
72 15578 quality touristic quality touristic 2 2 4
74 1667 given material given material 1 4 5
74 9202 given material given material 5 3 8
74 15809 given material given material 8 6 14
78 15822 regional transport regional transport 2 3 5
85 4970 new practical new practical 5 6 11
8

742 2277 given industrial given industrial 2 2 4
752 5357 opportunity environmental opportunity environmental 2 1 3
755 13359 route environmental route environmental 3 4 7
759 3176 question environmental question environmental 2 2 4
762 11856 new specialist new specialist 2 2 4
773 13116 application project application project 5 4 9
778 13545 given determined given determined 6 3 9
787 1777 new program new program 5 6 11
788 9014 degree touristic degree touristic 4 1 5
789 7101 population design population design 5 6 11
799 5903 design situation design situation 7 5 12
799 12593 design situation design situation 12 5 17
801 1073 source new source new 11 2 13
812 13546 point touristic point touristic 4 1 5
814 1990 main piece main piece 10 10 20
818 2735 new environment new environment 32 7 39
820 3852 view material view material 1 4 5
823 14364 tourist extreme tourist extreme 1 1 2
824 3724 new foundation new foundation 21 9 30
825 12132 tangible technology tangible technology 2 3 5
83

...

8001 8576 way touristic way touristic 5 4 9
8009 12097 new existing new existing 9 3 12
8015 12919 design Russian design Russian 0 0 0
8025 10416 usage design usage design 11 13 24
8042 9939 technology extreme technology extreme 9 1 10
8042 13579 technology extreme technology extreme 10 3 13
8042 15194 technology extreme technology extreme 13 9 22
8044 15196 solution extreme solution extreme 1 3 4
8059 11005 action ritual action ritual 5 1 6
8065 11757 design meaning design meaning 0 0 0
8076 9814 given school given school 10 3 13
8086 13576 Siberia touristic Siberia touristic 1 1 2
8095 14792 touristic function touristic function 2 2 4
8098 11339 material character material character 1 5 6
8108 13331 context ritual context ritual 7 10 17
8140 13258 given shape given shape 2 7 9
8141 13324 story design story design 4 1 5
8150 13797 human extreme human extreme 25 52 77
8161 9506 design industrial design industrial 15 6 21
8161 15837 design industrial design industrial 21 11 32
8167 1561

9301 15867 own extreme own extreme 6 3 9
9312 15711 necessity touristic necessity touristic 2 3 5
9331 15237 given educational given educational 6 4 10
9348 10527 approach application approach application 2 6 8
9352 13276 different state different state 2 1 3
9369 14486 given process given process 9 7 16
9396 13325 new result new result 13 5 18
9401 11537 time design time design 0 0 0
9402 11975 given scientific given scientific 4 6 10
9403 12710 material means material means 5 2 7
9405 14448 context extreme context extreme 7 1 8
9408 10015 development extreme development extreme 3 8 11
9412 13548 designer meaning designer meaning 2 1 3
9420 13316 given story given story 1 2 3
9439 10668 question design question design 0 0 0
9449 11918 designer design designer design 9 15 24
9454 14997 new part new part 7 1 8
9464 12480 side extreme side extreme 5 4 9
9468 15894 every design every design 7 3 10
9472 14397 given necessary given necessary 2 10 12
9484 10418 community environmental commun

11021 14284 new social new social 5 7 12
11066 13393 ritual system ritual system 9 5 14
11067 14850 content transport content transport 2 1 3
11082 12916 given ecology given ecology 1 2 3
11106 12900 principle design principle design 45 45 90
11110 11743 day application day application 1 1 2
11120 12868 world new world new 18 6 24
11122 15781 population transport population transport 2 1 3
11162 14740 fly-in-fly-out  different fly-in-fly-out  different 1 1 2
11202 11455 individual touristic individual touristic 9 5 14
11219 13654 adaptation new adaptation new 14 3 17
11226 15610 aspect different aspect different 2 6 8
11228 14440 day main day main 1 1 2
11237 12070 new main new main 0 0 0
11260 12621 material natural material natural 6 5 11
11269 15445 question transport question transport 2 1 3
11287 15249 very technology very technology 5 3 8
11295 15175 main principle main principle 13 14 27
11312 12237 indigenous new indigenous new 16 3 19
11365 12877 design existence design existe

14320 14991 new extreme new extreme 0 0 0
14361 15833 touristic element touristic element 2 1 3
14414 15077 day design day design 0 0 0
14416 15035 scientific application scientific application 4 1 5
14478 15740 another/different technology another/different technology 2 9 11
14506 15934 region transport region transport 9 8 17
14507 15030 ritual degree ritual degree 4 4 8
14509 15954 activity material activity material 3 4 7
14519 15265 different existing different existing 2 1 3
14540 15688 position design position design 7 3 10
14608 14728 application content application content 3 3 6
14631 15925 design content design content 0 0 0
14635 15031 main series main series 3 5 8
14661 15316 main value main value 2 2 4
14668 15719 day new day new 4 1 5
14808 15746 existence extreme existence extreme 5 10 15
15061 15929 application aim application aim 4 8 12
15078 15088 new transport new transport 0 0 0
15078 15157 new transport new transport 0 0 0
15080 15084 opportunity meaning opportunit

,translate_w1,translate_w2,value,duplicates
0,existence,character,1,False
1,high,technology,3,True
2,individual,environmental,4,False
3,opportunity,technological,4,False
4,complex,experience,2,False
...,...,...,...,...
15993,ornamental,specifics,2,False
15994,side,aim,1,False
15995,example,extreme,0,True
15996,place,contemporary,5,False


In [6]:
# del all duplicates  test_d.iloc[i, 2] == 0
#test_d[test_d.iloc[0, 2] != 0]
test_d.iloc[0, 2]
test_nd = test_d[test_d['value'] != 0]
test_nd

display(test_nd[test_nd.iloc[:,0:2].duplicated(keep = False)])
test_nd

NameError: name 'test_d' is not defined

In [98]:
'''3.
try to remove cycle cases like ['design', 'design']
''' 

test_nd['d2'] = 0
for i in range(len(foo)):
    for j in range(0,len(test_nd)):
        if list(test_nd.iloc[j,0:2]) == [foo.iloc[i,2], foo.iloc[i,2]]: 
            test_nd.iloc[j,3] = 1
            print(i, j, foo.iloc[i,2])
print(test_nd[test_nd['d2'] == 1])  

len(test_nd)
test_nd_uc = test_nd[test_nd['d2'] != 1]
len(test_nd_uc)


6 9674 design
9 9674 design
10 4797 new
17 9674 design
26 6509 transport
39 6509 transport
40 5130 extreme
44 13038 material
47 6856 given
50 3222 touristic
71 11541 ritual
72 4797 new
78 12692 environmental
88 4270 technology
90 1702 main
103 8093 different
104 1702 main
118 3222 touristic
121 12692 environmental
129 9241 application
132 8093 different
139 4270 technology
146 9241 application
153 13038 material
160 5130 extreme
164 11541 ritual
166 6856 given
186 3232 meaning
191 3232 meaning
        translate_w1   translate_w2  value  d2  (9674, d2)
1726            main           main      4   1           1
3323       touristic      touristic      6   1           1
3333         meaning        meaning      6   1           1
4427      technology     technology      8   1           1
4997             new            new     27   1           1
5355         extreme        extreme      5   1           1
6859       transport      transport     25   1           1
7241           given         

In [100]:
print(len(test_nd))
test_nd_uc = test_nd[test_nd['d2'] != 1]
len(test_nd_uc)

14265


14251

In [101]:
# output
test_nd_uc = test_nd_uc.iloc[:,[0, 1, 2]]
test_nd_uc.to_csv('210216_arctic_ruen_test_merged_e2e_uc.csv', sep=',', header = False, index = False)
# foo_clean = foo.iloc[:,[2, 1]]
# foo_clean.to_csv('210216_arctic_foo_ruen.csv', sep=',', header = False, index = False)

In [72]:
'''5. 
merge in foo '''

foo_d = pd.read_csv('210216_arctic_foo_ruen.csv', names = ["w1", 'value'])

foo_d["duplicates"] = foo_d.iloc[:,0].duplicated(keep = False)
display(foo_d[foo_d.iloc[:,0].duplicated(keep = False)])

for i in range(0,len(foo_d)):
    if foo_d.iloc[i, 2] == True:
        # нужно найти все дубликаты, сложить числа в ячейке value
        # записать их к первому дубликату, а остальные дубликаты удалить
        t = foo_d.iloc[i,0]
        for j in range(i+1,len(foo_d)):
            if foo_d.iloc[j, 2] == True:
                if (list(t) == list(foo_d.iloc[j,0])): 
                    print(i, j, foo_d.iloc[i,0], foo_d.iloc[j,0], foo_d.iloc[i,1], foo_d.iloc[j,1], foo_d.iloc[i, 1] + foo_d.iloc[j, 1])
                    foo_d.iloc[i, 1] = foo_d.iloc[i, 1] + foo_d.iloc[j, 1]
                    foo_d.iloc[j, 1] = 0
foo_d 

,w1,value,duplicates
6,design,347,True
9,design,312,True
10,new,305,True
17,design,212,True
26,transport,179,True
39,transport,152,True
40,extreme,149,True
44,material,141,True
47,given,139,True
50,touristic,135,True


6 9 design design 347 312 659
6 17 design design 659 212 871
9 17 design design 0 0 0
10 72 new new 305 108 413
26 39 transport transport 179 152 331
40 160 extreme extreme 149 56 205
44 153 material material 141 59 200
47 166 given given 139 55 194
50 118 touristic touristic 135 74 209
71 164 ritual ritual 108 55 163
78 121 environmental environmental 102 73 175
88 139 technology technology 93 66 159
90 104 main main 91 81 172
103 132 different different 82 69 151
129 146 application application 70 63 133
186 191 meaning meaning 50 49 99


,w1,value,duplicates
0,North,546,False
1,human,498,False
2,culture,485,False
3,northern,471,False
4,research,398,False
...,...,...,...
195,expedition,49,False
196,relaxation,48,False
197,educational,48,False
198,Russian,48,False


In [74]:
# del all duplicates  test_d.iloc[i, 2] == 0

foo_nd = foo_d[foo_d['value'] != 0]
foo_nd = foo_nd.iloc[:,[0, 1]]
foo_nd.to_csv('210216_arctic_foo_ruen_nodubl.csv', sep=',', header = False, index = False)

In [16]:
# sort foo 
foo = pd.read_csv('210215_arctic_foo_ruen_nodubl.csv', sep=',', names = ["w1", 'value'])
foo = foo.sort_values(by = ['value'], ascending = False)
foo.to_csv('210217_arctic_foo_ruen_nodubl.csv', sep=',', header = False, index = False)